<a href="https://colab.research.google.com/github/Samgomes2510/dashborad-ecommerce/blob/main/2AgrecacoesBasicas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install spark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.0/41.0 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for spark: filename=spark-0.2.1-py3-none-any.whl size=58748 sha256=71cc675fb3973efe43f3f447435935cec377595584680e4b797c1bf536557ca9
  Stored in directory: /root/.cache/pip/wheels/67/c2/7c/a53325365fba358ffff35af84a2e14cf88c18052f88acfa5f0
Successfully built spark


In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [3]:
from google.colab import files
uploaded = files.upload()

Saving avaliacoes_pedido.csv to avaliacoes_pedido.csv
Saving clientes.csv to clientes.csv
Saving itens_pedido.csv to itens_pedido.csv
Saving pagamentos_pedido.csv to pagamentos_pedido.csv
Saving pedidos.csv to pedidos.csv
Saving produtos.csv to produtos.csv
Saving vendedores.csv to vendedores.csv


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
spark = SparkSession.builder.getOrCreate()

In [6]:
clientes = spark.read.csv("clientes.csv", header=True, inferSchema=True)
itens_pedido = spark.read.csv("itens_pedido.csv", header=True, inferSchema=True)
pagamentos_pedido = spark.read.csv("pagamentos_pedido.csv", header=True, inferSchema=True)
pedidos = spark.read.csv("pedidos.csv", header=True, inferSchema=True)

In [7]:
clientes.show()
pagamentos_pedido.show()
pedidos.show()
itens_pedido.show()

+--------------------+--------------------+-----------+--------------------+--------------+
|          id_cliente|    id_unico_cliente|cep_cliente|      cidade_cliente|estado_cliente|
+--------------------+--------------------+-----------+--------------------+--------------+
|06b8999e2fba1a1fb...|861eff4711a542e4b...|      14409|              franca|            SP|
|18955e83d337fd6b2...|290c77bc529b7ac93...|       9790|sao bernardo do c...|            SP|
|4e7b3e00288586ebd...|060e732b5b29e8181...|       1151|           sao paulo|            SP|
|b2b6027bc5c5109e5...|259dac757896d24d7...|       8775|     mogi das cruzes|            SP|
|4f2d8ab171c80ec83...|345ecd01c38d18a90...|      13056|            campinas|            SP|
|879864dab9bc30475...|4c93744516667ad3b...|      89254|      jaragua do sul|            SC|
|fd826e7cf63160e53...|addec96d2e059c80c...|       4534|           sao paulo|            SP|
|5e274e7a0c3809e14...|57b2a98a409812fe9...|      35182|             timoteo|    

In [8]:
clientes.agg(countDistinct(col('estado_cliente')).alias('qtd_estados_distintos')).show()

+---------------------+
|qtd_estados_distintos|
+---------------------+
|                   27|
+---------------------+



In [9]:
pagamentos_pedido.groupBy('tipo_pagamento').agg(
    avg('valor_pagamento').alias('Pagamento médio'),
    var_samp('valor_pagamento').alias('Pagamento variância')
).show()


+--------------+------------------+-------------------+
|tipo_pagamento|   Pagamento médio|Pagamento variância|
+--------------+------------------+-------------------+
|        boleto| 145.0344354023452| 45616.869821012544|
|   not_defined|               0.0|                0.0|
|   credit_card|163.31902063935797|  49336.98820414979|
|       voucher| 65.70335411255421| 13344.682202516573|
|    debit_card|142.57017004578165| 60414.395995128165|
+--------------+------------------+-------------------+



In [11]:
pagamentos_pedido.groupBy('tipo_pagamento').agg(
    format_number(avg('valor_pagamento'),2).alias('Pagamento médio'),
    format_number(var_samp('valor_pagamento'),2).alias('Pagamento variância')
).show()


+--------------+---------------+-------------------+
|tipo_pagamento|Pagamento médio|Pagamento variância|
+--------------+---------------+-------------------+
|        boleto|         145.03|          45,616.87|
|   not_defined|           0.00|               0.00|
|   credit_card|         163.32|          49,336.99|
|       voucher|          65.70|          13,344.68|
|    debit_card|         142.57|          60,414.40|
+--------------+---------------+-------------------+



In [12]:
pedidos.printSchema()

root
 |-- id_pedido: string (nullable = true)
 |-- id_cliente: string (nullable = true)
 |-- status_pedido: string (nullable = true)
 |-- data_compra_pedido: timestamp (nullable = true)
 |-- data_aprovacao_pedido: timestamp (nullable = true)
 |-- data_envio_transportadora: timestamp (nullable = true)
 |-- data_entrega_cliente: timestamp (nullable = true)
 |-- data_estimada_entrega: timestamp (nullable = true)



In [19]:
pedidos_ordenados = pedidos.orderBy('data_compra_pedido')


In [20]:
data_compra_pedido_df = pedidos_ordenados.groupBy('status_pedido').agg(
    first('data_compra_pedido').alias('primeira_compra'),
    last('data_compra_pedido').alias('ultima_compra')
)
data_compra_pedido_df.show()

+-------------+-------------------+-------------------+
|status_pedido|    primeira_compra|      ultima_compra|
+-------------+-------------------+-------------------+
|      shipped|2016-09-04 21:15:19|2018-09-03 09:06:57|
|     canceled|2016-09-05 00:15:34|2018-10-17 17:30:18|
|     approved|2017-02-06 20:18:17|2017-04-25 01:25:34|
|     invoiced|2016-10-04 13:02:10|2018-08-14 18:45:08|
|      created|2017-11-06 13:12:34|2018-02-09 17:21:04|
|    delivered|2016-09-15 12:16:38|2018-08-29 15:00:37|
|  unavailable|2016-10-05 14:16:28|2018-08-21 12:21:00|
|   processing|2016-10-05 22:44:13|2018-07-23 18:03:03|
+-------------+-------------------+-------------------+



In [22]:
pedidos.withColumn("compra_anual", year("data_compra_pedido")).groupBy("compra_anual").count().orderBy(asc("compra_anual")).show()

+------------+-----+
|compra_anual|count|
+------------+-----+
|        2016|  329|
|        2017|45101|
|        2018|54011|
+------------+-----+



In [29]:
itens_pedido.groupBy('id_pedido').agg(
    sum(col("preco") + col("valor_frete")), # Changed 'frete' to 'valor_frete'
    sum(col("preco") + col("valor_frete")).alias("valor_total_pedido"),
    format_number(sum(col("preco") + col("valor_frete")),2).alias("valor_total_pedido_formatado")
).show()

+--------------------+--------------------------+------------------+----------------------------+
|           id_pedido|sum((preco + valor_frete))|valor_total_pedido|valor_total_pedido_formatado|
+--------------------+--------------------------+------------------+----------------------------+
|014405982914c2cde...|         78.42999999999999| 78.42999999999999|                       78.43|
|019886de8f385a39b...|                     188.4|             188.4|                      188.40|
|01a6ad782455876aa...|                     50.09|             50.09|                       50.09|
|01d907b3e209269e1...|        169.76000000000002|169.76000000000002|                      169.76|
|028dc52e12ddda803...|                     61.72|             61.72|                       61.72|
|036dd381dfb3ec75e...|                     69.14|             69.14|                       69.14|
|03ebfa9712b7dbc70...|                     55.78|             55.78|                       55.78|
|0420da8d50a378401..

In [30]:
spark.stop()